<h2 style="text-align:center;font-size:200%;">
    <b>Model Explainability in Industrial Image Detection</b>
</h2>
<h3  style="text-align:center;">Keywords : 
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Image Classification</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Data Augmentation</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">CNN</span><br><br>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Model Explanation</span>
    <span style="border-radius:7px;background-color:limegreen;color:white;padding:7px;">Error Analysis</span>
</h3>

<hr>

<a id='top'></a>
<h2 style="font-size:150%;"><span id='top'>Table of Contents</span></h2>
<blockquote>
    <ol>
        <li><a href="#Overview">Overview</a></li>
        <ul>
            <li><a href="#Task-Detail">Task Detail</a></li>
            <li><a href="#About-Dataset">About Dataset</a></li>
        </ul>
        <li><a href="#Import-libraries">Import libraries</a></li>
        <li><a href="#Load-the-dataset">Load the dataset</a></li>
        <li><a href="#Pre-Processing">Pre-Processing</a></li>
        <ul>
            <li><a href="#What-is-Data-Augmentation?">What is Data Augmentation?</a></li>
            <li><a href="#Execute-Data-Augmentation">Execute Data Augmentation</a></li>
        </ul>
        <li><a href="#Modeling">Modeling</a></li>
        <ul>
            <li><a href="#Model-Settings">Model Settings</a></li>
            <li><a href="#Build-Model">Build Model</a></li> 
            <li><a href="#Model-Performance">Model Performance</a></li>
            <li><a href="#Predict-on-Some-Images">Predict on Some Images</a></li>
        </ul>
        <li><a href="#Conclusion">Conclusion</a></li>
    </ol>
</blockquote>

# <div style="text-align: left; background-color: mediumseagreen; color: white; padding: 10px; line-height:1;border-radius:10px"><span id='Overview'>Overview</span></div>
 
<h2 style="font-size:150%;"><span id='Task-Detail'>Task Details</span></h2>

* Select or create a dataset that includes images of industrial equipment labelled
as 'defective' or 'non-defective', with additional labels for the type of defect in
defective images.

* Train a machine learning model to classify images into the two main categories.

* Evaluate the model's performance using classification metrics such as accuracy,
precision, and recall.


<h2 style="font-size:150%;"><span id='About-Dataset'>About Dataset</span></h2>
This dataset provides image data of impellers for submersible pumps.<br/><br/>
<table border="0">
     <tr style="background-color: white !important;">
        <td>
            <img src="https://static.turbosquid.com/Preview/2020/06/07__08_34_27/11R131.JPGB3B4468C-B515-4E11-92F7-4CA67966DB2BZoom.jpg" width="300">
            <figcaption style="text-align:center">Submersible Pump</figcaption>
        </td>
        <td>
            <img src="https://5.imimg.com/data5/WI/KC/MY-6121640/submersible-pump-impeller-500x500.jpg" width="300">
            <figcaption style="text-align:center">Impeller</figcaption>
        </td>
    </tr>
 </table><br/>
The image data is labeled with <b>ok(normal)</b> for non-defective equipment and <b>def(defect/anomaly)</b> for defective equipment.

<li>Link to DataSet :- <a href="https://github.com/Us2id/Classification-of-equipment-as-defective-and-non-defective-.-"> Link</a></li>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# <div style="text-align: left; background-color: mediumseagreen; color: white; padding: 10px; line-height:1;border-radius:10px"><span id='Import-libraries'>Import libraries</span></div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import json

from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras import backend
from sklearn.metrics import confusion_matrix, classification_report

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# <div style="text-align: left; background-color: mediumseagreen; color: white; padding: 10px; line-height:1;border-radius:10px"><span id='Load-the-dataset'>Load the dataset</span></div>

In [ ]:
train_path = 'C:\\Users\\usaid\\Downloads\\casting_data\\train'
test_path = 'C:\\Users\\usaid\\Downloads\\casting_data\\test'

In [ ]:
plt.figure(figsize=(10,8))
ok = plt.imread(train_path + '\\ok_front\\cast_ok_0_1.jpeg')
plt.subplot(1, 2, 1)
plt.axis('off')
plt.title("ok", weight='bold', size=20)
plt.imshow(ok,cmap='gray')
plt.show()

ng = plt.imread(train_path + '\\def_front\\cast_def_0_8.jpeg')
plt.subplot(1, 2, 2)
plt.axis('off')
plt.title("def", weight='bold', size=20)
plt.imshow(ng,cmap='gray')

plt.show()

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# <div style="text-align: left; background-color: mediumseagreen; color: white; padding: 10px; line-height:1;border-radius:10px"><span id='Pre-Processing'>Pre-Processing</span></div>
In training models for image classification, <b>Data Augmentation</b> techniques are needed to build more robust models.

<h2 style="font-size:150%;"><span id='What-is-Data-Augmentation?'>What is Data Augmentation?</span></h2>
<div class="alert alert-success" role="alert" style="border-radius:10px">
    <p>When training with image data without data augmentation, we simply need the specified number of data and create a mini-batch. When executing data augmentation, after acquiring the data, various augmentation techniques are applied to the image to create a new mini-batch. <br/>
    The main parameters of data augmentation techniques are as follows : </p>
    <ul>
        <li><b>rotation_range</b> : Rotate the image (ex. 50 -> rotate randomly in -50°~50°)</li>
        <li><b>zoom_range</b> : Zoom in/out on the image (ex. 0.5 -> zoom in/out randomly in 1-0.5~1+0.5)</li>
        <li><b>brightness_range</b> : Change the brightness (ex. [0.3,1.0] -> change randomly in [0.3,1.0])</li>
        <li><b>vertical_flip</b> : Flip the image upside down</li>
        <li><b>horizontal_flip</b> : Flip the image left or right</li>
        <li><b>height_shift_range</b> : Move the image up or down in parallel (ex. 0.3 -> move up/down randomly in [-0.3*Height, 0.3*Height])</li>
        <li><b>width_shift_range</b> : Move the image left or right in parallel (ex. 0.3 -> move left/right randomly in [-0.3*Width, 0.3*Width])</li>
        <li><b>rescale</b> : The image is normalized by multiplying each pixel value by a constant. (ex. 1/255 -> normalize the RGB value of each pixel between 0.0 and 1.0)</li>
    </ul>
</div>

In [ ]:
img = cv2.imread(train_path + '\\ok_front\\cast_ok_0_1.jpeg')
img_4d = img[np.newaxis]
plt.figure(figsize=(25,10))
generators = {"rotation":ImageDataGenerator(rotation_range=180), 
              "zoom":ImageDataGenerator(zoom_range=0.7), 
              "brightness":ImageDataGenerator(brightness_range=[0.2,1.0]), 
              "height_shift":ImageDataGenerator(height_shift_range=0.7), 
              "width_shift":ImageDataGenerator(width_shift_range=0.7)}

plt.subplot(1, 6, 1)
plt.title("Original", weight='bold', size=15)
plt.imshow(img)
plt.axis('off')
cnt = 2
for param, generator in generators.items():
    image_gen = generator
    gen = image_gen.flow(img_4d, batch_size=1)
    batches = next(gen)
    g_img = batches[0].astype(np.uint8)
    plt.subplot(1, 6, cnt)
    plt.title(param, weight='bold', size=15)
    plt.imshow(g_img)
    plt.axis('off')
    cnt += 1
plt.show()

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

<h2 style="font-size:150%;"><span id='Execute-Data-Augmentation'>Execute Data Augmentation</span></h2>
<div class="alert alert-success" role="alert" style="border-radius:10px">
    <p>In Keras, you can pass ImageDataGenerator class as a dataset when training a model, and it creates a mini-batch by randomly applying the parameters. Empirically, if the parameter is set to an extreme high/low value, <u>the image will be strongly converted and the training will be difficult to proceed.</u> In addition, it seems that fine adjustment of parameters is required for each target data or analysis objectives in order to proceed with training successfully.<br></p>
</div>

In [ ]:
image_gen = ImageDataGenerator(rescale=1/255, 
                               zoom_range=0.1, 
                               brightness_range=[0.9,1.0])

In [ ]:
image_shape = (300,300,1)
batch_size = 32

train_set = image_gen.flow_from_directory(train_path,
                                            target_size=image_shape[:2],
                                            color_mode="grayscale",
                                            classes={'def_front': 0, 'ok_front': 1},
                                            batch_size=batch_size,
                                            class_mode='binary',
                                            shuffle=True,
                                            seed=0)

test_set = image_gen.flow_from_directory(test_path,
                                           target_size=image_shape[:2],
                                           color_mode="grayscale",
                                           classes={'def_front': 0, 'ok_front': 1},
                                           batch_size=batch_size,
                                           class_mode='binary',
                                           shuffle=False,
                                           seed=0)

In [ ]:
train_set.class_indices

* class 0 : defect 
* class 1 : ok

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

# <div style="text-align: left; background-color: mediumseagreen; color: white; padding: 10px; line-height:1;border-radius:10px"><span id='Modeling'>Modeling</span></div>

<h2 style="font-size:150%;"><span id='Model-Settings'>Model-Settings</span></h2>
<div class="alert alert-success" role="alert" style="border-radius:10px">
    <p>The elements of the model are listed below.</p>
    <ul>
        <li><b>Sequential</b> : model container</li>
        <li><b>Conv2D</b> : convolutional layer for 2D images
            <ul>
                <li><b>filters</b> : number of filters
                    <ul>
                        <li>numbers such as <u>16, 32, 64, 128, 256 and 512</u> tend to be used, and there is a technique to increase the number of filter for the complicated task and decrease it for simple one.</li>
                    </ul>
                </li>
                <li><b>kernel_size</b> : filter size (width * height)
                    <ul>
                        <li>combinations of odd numbers such as <u>3x3, 5x5, 7x7</u> tend to be used.</li>
                    </ul>
                </li>
                <li><b>strides</b> : window size used for convolution</li>
                <li><b>input_shape</b> : size of input images (width/height, color channel)
                    <ul>
                        <li>if you input color images as it is, the model will need convolutions for 3 RGB channels, which will increase the amount of calculation(graysclaed images need less calculations).</li>
                    </ul>
                </li>
                <li><b>activation</b> : activation function</li>
                <li><b>padding</b> : adjust the size of the layer output. When set to 'same', the pixels are filled with 0 so that the input and output sizes are the same.</li>
            </ul>
        </li>
        <li><b>MaxPooling2D</b> : pooling layer for 2D images
            <ul>
                <li><b>pool_size</b> : specify width/height range and extract the largest pixel in this range to downscale the input</li>
                <li><b>strides</b> : window size used for pooling</li>
            </ul>
        </li>
        <li><b>Flatten</b> : convert input to linear vector</li>
        <li><b>Dropout</b> : apply dropout and randomly set the input to the unit to 0 to prevent overfitting when updating weights
            <ul>
                <li><b>rate</b> : ratio of dropping the input to the unit</li>
            </ul>
        </li>
        <li><b>Dense</b> : fully connected layer
            <ul>
                <li><b>units</b> : number of dimensions of output</li>
                <li><b>activation</b> : activation function（binary classification : <u>sigmoid</u>, other objectives : <u>softmax</u>） </li>
            </ul>
        </li>
    </ul>
</div>

In [ ]:
backend.clear_session()
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(7,7), strides=2, input_shape=image_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=1, input_shape=image_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=1, input_shape=image_shape, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(units=224, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

The figure below shows the model architecture.

In [ ]:
plot_model(model, show_shapes=True, expand_nested=True, dpi=60)

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

<h2 style="font-size:150%;"><span id='Build-Model'>Build-Model</span></h2>
<div class="alert alert-success" role="alert" style="border-radius:10px">
    Build settings:  
    <ul>
        <li><b>EarlyStopping</b> : Conditions to stop training early</li>
            <ul>
                <li>ex. validation loss not improved continuously in 2 epochs</li>
            </ul>
        <li><b>ModelCheckpoint</b> : Model saving settings for each epoch</li>
    </ul>
</div>

In [ ]:
model_save_path = 'casting_product_detection.hdf5'
early_stop = EarlyStopping(monitor='val_loss',patience=2)
checkpoint = ModelCheckpoint(filepath=model_save_path, verbose=1, save_best_only=True, monitor='val_loss')

In [ ]:
n_epochs = 20
results = model.fit_generator(train_set, epochs=n_epochs, validation_data=test_set, callbacks=[early_stop,checkpoint])

In [ ]:
model_history = { i:list(map(lambda x: float(x), j)) for i,j in results.history.items() }
with open('model_history.json', 'w') as f:
    json.dump(model_history, f, indent=4)

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

<h2 style="font-size:150%;"><span id='Model-Performance'>Model Performance</span></h2>

In [ ]:
losses = pd.DataFrame(model_history)
losses.index = map(lambda x : x+1, losses.index)
losses.head(3)

Since the loss at the time of training and validation are steadily decreasing for each epoch, and the accuracy at the time of training and the validation are steadily increasing, it can be said that the training is generally successful.

In [ ]:
g = hv.Curve(losses.loss, label='Training Loss') * hv.Curve(losses.val_loss, label='Validation Loss') \
    * hv.Curve(losses.accuracy, label='Training Accuracy') * hv.Curve(losses.val_accuracy, label='Validation Accuracy')
g.opts(opts.Curve(xlabel="Epochs", ylabel="Loss / Accuracy", width=700, height=400,tools=['hover'],show_grid=True,title='Model Evaluation')).opts(legend_position='bottom')

In [ ]:
pred_probability = model.predict_generator(test_set)
predictions = pred_probability > 0.5

plt.figure(figsize=(10,6))
plt.title("Confusion Matrix", size=20, weight='bold')
sns.heatmap(
    confusion_matrix(test_set.classes, predictions),
    annot=True,
    annot_kws={'size':14, 'weight':'bold'},
    fmt='d',
    xticklabels=['Defect', 'OK'],
    yticklabels=['Defect', 'OK'])
plt.tick_params(axis='both', labelsize=14)
plt.ylabel('Actual', size=14, weight='bold')
plt.xlabel('Predicted', size=14, weight='bold')
plt.show()

In [ ]:
print(classification_report(test_set.classes, predictions, digits=3))

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>

<h2 style="font-size:150%;"><span id='Predict-on-Some-Images'>Predict on Some Images</span></h2>
Select images and apply it to the model.

In [ ]:
test_cases = ['\\ok_front\\cast_ok_0_10.jpeg', '\\ok_front\\cast_ok_0_1026.jpeg', '\\ok_front\\cast_ok_0_1031.jpeg', '\\ok_front\\cast_ok_0_1121.jpeg',
              '\\ok_front\\cast_ok_0_1144.jpeg','\\def_front\\cast_def_0_1059.jpeg', '\\def_front\\cast_def_0_108.jpeg', '\\def_front\\cast_def_0_1153.jpeg',
              '\\def_front\\cast_def_0_1238.jpeg', '\\def_front\\cast_def_0_1269.jpeg']
plt.figure(figsize=(20,8))
for i in range(len(test_cases)):
    img_pred = cv2.imread(test_path + test_cases[i], cv2.IMREAD_GRAYSCALE)
    img_pred = img_pred / 255 # rescale
    prediction = model.predict(img_pred.reshape(1, *image_shape))
    
    img = cv2.imread(test_path + test_cases[i])
    label = test_cases[i].split("_")[0]
    
    plt.subplot(2, 5, i+1)
    parts = test_cases[i].split('\\')
    plt.title(f"{parts[2]}\n Actual Label : {label}", weight='bold', size=12)
    # Predicted Class : defect
    if (prediction < 0.5):
        predicted_label = "def"
        prob = (1-prediction.sum()) * 100
    # Predicted Class : OK
    else:
        predicted_label = "ok"
        prob = prediction.sum() * 100
        
    cv2.putText(img=img, text=f"Predicted Label : {predicted_label}", org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(255, 0, 255), thickness=2)
    cv2.putText(img=img, text=f"Probability : {'{:.3f}'.format(prob)}%", org=(10, 280), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.7, color=(0, 255, 0), thickness=2)
    plt.imshow(img,cmap='gray')
    plt.axis('off')

plt.show()

# <div style="text-align: left; background-color: mediumseagreen; color: white; padding: 10px; line-height:1;border-radius:10px"><span id='Conclusion'>Conclusion</span></div>

<div class="alert alert-success" role="alert" style="border-radius:10px">
    <ul>
        <li><b>Data augmentation</b> proces can be easily incorporated into training process by using ImageDataGenerator.</li>
        <li>In data augmentation process, we should avoid excessive conversion and may require subtle adjustments depending on the dataset.</li>
        <li>According to the result of model interpretation, it was found that <b>the scratches/holes on the surface of the products and the unevenness around the products</b> are regarded as the important features of defective products.</li>
        <li>Since we successfully built a model with relatively high accuracy, it is considered possible to incorporate the model into the camera of the inspection line and proceed with the automation of inspection.
        </li>
        <li>Link to DataSet :- <a href="https://github.com/Us2id/Classification-of-equipment-as-defective-and-non-defective-.-"> Link</a>
        </li>
    </ul>
</div>

<button class="label alert-success" style="border-radius:10px;padding:10px;font-size:18px"><a href="#top" style="color:green;"><b>Table of Contents</b></a></button>